In [1]:
import requests,pprint,json
try:
  no_of_records = 500
  baseurl="https://zucwflxqsxrsmwseehqvjmnx2u0cdigp.lambda-url.ap-south-1.on.aws"
  endpoint = "/mentorskool/v1/sales?offset=0&limit=100"
  records = []
  for i in range(5):
    response = requests.get(baseurl+endpoint,headers = {"access_token":"fe66583bfe5185048c66571293e0d358"})
    res_data = response.json()
    records.extend(res_data['data'])
    endpoint = res_data['next']


  pprint.pprint(records)
except IndexError:
  print("invalid indexing reached")
except requests.exceptions.HTTPError as i:
  print(i)
except requests.exceptions.RequestException as err:
  print(err)

Streaming output truncated to the last 5000 lines.
                       'VendorID': 'VEN05'}},
  'product': {'category': 'Office Supplies',
              'colors': 'Blue',
              'date_added': '2017-01-12',
              'manufacturer': 'Stacy Adams',
              'product_id': 'OFF-AR-10002399',
              'product_name': 'Dixon Prang Watercolor Pencils, 10-Color Set '
                              'with Brush',
              'product_photos_qty': 2,
              'sizes': '9.5',
              'sub_category': 'Art',
              'upc': 718000000000,
              'weight': 'null'},
  'profit_amt': 6.9864,
  'qty': 4,
  'sales_amt': 17.04},
 {'discount': 0.0,
  'id': 4813,
  'order': {'customer': {'address': {'city': 'los angeles',
                                     'country': 'United States',
                                     'region': 'West',
                                     'state': 'california',
                                     'zip_code': 90008},
       

In [2]:
import pandas as pd
df = pd.json_normalize(records)

In [3]:
final_df = df[['id','sales_amt','qty','discount','profit_amt','order.order_id','order.order_purchase_date','order.order_status','order.order_delivered_customer_date','order.order_estimated_delivery_date','product.product_id','product.colors','product.category','order.customer.customer_id','order.customer.customer_name','order.vendor.VendorID']].copy()

In [4]:
final_df.rename(columns ={'order.order_id':'order_id','order.order_purchase_date':'order_purchase_date','order.order_status':'order_status',
                          'order.order_delivered_customer_date':'order_delivered_customer_date','order.order_estimated_delivery_date':'order_estimated_delivery_date',
                          'product.product_id':'product_id','product.colors':'colors','product.category':'category','order.customer.customer_id':'customer_id',
                          'order.customer.customer_name':'customer_name','order.vendor.VendorID':'vendorid'},inplace =True)

In [5]:
final_df.head()

,id,sales_amt,qty,discount,profit_amt,order_id,order_purchase_date,order_status,order_delivered_customer_date,order_estimated_delivery_date,product_id,colors,category,customer_id,customer_name,vendorid
0,4102,2.0000,0,72.948,null,US-2017-102288,2017-11-24 15:58:00,delivered,2017-12-15 14:58,2017-12-21,OFF-AP-10002684,Blue,Office Supplies,ZC-21910,Zuschuss Carroll,VEN04
1,2698,261.9600,2,0.000,41.9136,CA-2014-145317,2018-07-11 19:46:00,delivered,2018-07-24 14:58,2018-07-27,FUR-BO-10001798,Pink,Furniture,SM-20320,Sean Miller,VEN03
2,6827,731.9400,3,0.000,219.582,CA-2016-118689,2018-05-15 20:24:00,delivered,2018-06-04 14:42,2018-06-18,FUR-CH-10000454,Pink,Furniture,TC-20980,Tamara Chand,VEN01
3,8154,14.6200,2,0.000,6.8714,CA-2017-140151,2018-02-28 13:40:00,delivered,2018-04-03 23:31,2018-03-28,OFF-LA-10000240,Pink,Office Supplies,RB-19360,Raymond Buch,VEN02
4,2624,957.5775,5,0.450,-383.031,CA-2017-127180,2018-01-07 21:45:00,delivered,2018-01-30 20:51,2018-02-07,FUR-TA-10000577,Blue,Furniture,TA-21385,Tom Ashbrook,VEN01


In [6]:
final_df.replace('null',None,inplace= True)

In [7]:
final_df.isnull().sum()

id                                0
sales_amt                         0
qty                               0
discount                          0
profit_amt                        1
order_id                          0
order_purchase_date               0
order_status                      0
order_delivered_customer_date    12
order_estimated_delivery_date     2
product_id                        0
colors                            0
category                          0
customer_id                       0
customer_name                     0
vendorid                          0
dtype: int64

In [8]:
final_df[final_df.duplicated()].shape

(0, 16)

In [9]:
final_df['order_purchase_date'] =pd.to_datetime(final_df['order_purchase_date'])
final_df['order_delivered_customer_date'] = pd.to_datetime(final_df['order_delivered_customer_date'])
final_df['order_estimated_delivery_date'] = pd.to_datetime(final_df['order_estimated_delivery_date'])

In [10]:
final_df.dtypes

id                                        int64
sales_amt                               float64
qty                                       int64
discount                                float64
profit_amt                               object
order_id                                 object
order_purchase_date              datetime64[ns]
order_status                             object
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
product_id                               object
colors                                   object
category                                 object
customer_id                              object
customer_name                            object
vendorid                                 object
dtype: object

In [11]:
final_df['day_of_order'] = final_df['order_purchase_date'].dt.weekday
final_df['day_label'] = final_df['day_of_order'].apply(lambda x : 'weekend' if (x==5) or (x==6) else 'weekday')

In [13]:
final_df[final_df['day_label']=='weekend'].shape[0]

98

In [14]:
final_df.groupby('day_label')['sales_amt'].sum()

day_label
weekday    108528.1666
weekend     20830.8495
Name: sales_amt, dtype: float64

In [15]:
final_df.groupby('category')['sales_amt'].sum()

category
Furniture          45847.6101
Office Supplies    31469.8750
Technology         52041.5310
Name: sales_amt, dtype: float64